# Анализ базы данных

**Цель:** анализ базы данных для формулировки ценностного предложения нового продукта.

**Заказчик:** стартап, занимающийся разработкой приложения для любителей книг.

**Источник данных:** крупный сервис для чтения книг по подписке.  

**План:**
1. Импорт и обзор данных
2. Написание необходимых SQL-запросов для оценки интересующих параметров
3. Формулировка выводов

## Импорт и обзор таблиц

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
#создадим функцию для упрощения вывода результатов SQL-запросов
def sql(table_name):
    return pd.io.sql.read_sql(table_name, con = engine)

#и функцию для вывода информации о таблицах
def table(table_name):
    name = pd.io.sql.read_sql(table_name, con = engine)
    print (name.info())
    display (name.head())
    print('Количество дубликатов:', name.duplicated().sum())

Для начала оценим таблицы с данными, которые у нас есть.

In [3]:
#таблица books
table('SELECT * FROM books')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Количество дубликатов: 0


In [4]:
#таблица authors
table('SELECT * FROM authors')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Количество дубликатов: 0


In [5]:
#таблица publishers
table('SELECT * FROM publishers')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Количество дубликатов: 0


In [6]:
#таблица ratings
table('SELECT * FROM ratings')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Количество дубликатов: 0


In [7]:
#таблица reviews
table('SELECT * FROM reviews')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Количество дубликатов: 0


У нас имеется следующая информация:
* **1000** книг
* **636** авторов
* **340** издательств
* **6456** пользовательских оценок
* **2793** пользовательских обзора

Пропуски и явные дубликаты в данных отсутствуют.

## Анализ базы данных

In [8]:
#посчитаем, сколько книг вышло после 1 января 2000 года
sql(
    "SELECT count(book_id) AS books_count\
    FROM books\
    WHERE publication_date > '2000-01-01'"
)

,books_count
0,819


Из 1000 представленных книг **819** была выпущена после 2000 года, это больше 81%.

In [9]:
#посчитаем количество обзоров и среднюю оценку для каждой книги
sql(
    "\
    SELECT k.title,\
            k.count,\
            ROUND(AVG(t.rating),2)\
    FROM (SELECT b.title, b.book_id,\
           COUNT(r.review_id)\
    FROM books AS b LEFT JOIN reviews AS r ON b.book_id = r.book_id\
    GROUP BY b.title, b.book_id) AS k JOIN ratings AS t ON k.book_id = t.book_id\
    GROUP BY k.title, k.count\
    ORDER BY count\
    "
)

,title,count,round
0,Disney's Beauty and the Beast (A Little Golden...,0,4.00
1,Leonardo's Notebooks,0,4.00
2,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
3,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
4,Essential Tales and Poems,0,4.00
...,...,...,...
995,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
996,The Road,6,3.77
997,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
998,The Hobbit or There and Back Again,6,4.13


Взаимосвязи между количеством обзоров и средней пользовательской оценкой не обнаружено: даже при отсутствии обзоров оценка может максимальной (5.00), или чуть выше средней (3.66) несмотря на максимальное количество обзоров. 

In [10]:
#определим, какое издательство выпустило больше всего книг (толще 50 страниц)
sql(
    'SELECT publisher\
    FROM (SELECT p.publisher,\
            COUNT(b.book_id) AS books_count\
    FROM books AS b LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id\
    WHERE b.num_pages > 50\
    GROUP BY p.publisher\
    ORDER BY COUNT(b.book_id) DESC\
    LIMIT 1) AS a'
)

,publisher
0,Penguin Books


Самым активным оказалось издательство **Penguin Books** с 42 выпущенными книгами.

In [11]:
#найдем автора с самой высокой средней оценкой книг (для книг, у которых 50 и более оценок)
sql(
    'WITH cnt AS (SELECT book_id\
                    FROM(SELECT b.book_id,\
                        COUNT(rating_id) AS rating_count\
                        FROM books AS b JOIN ratings AS r ON b.book_id = r.book_id\
                        GROUP BY b.book_id) AS f\
                        WHERE rating_count >=50)\
    SELECT author, avg_rating\
    FROM (SELECT author_id,\
                 ROUND(AVG(rating), 2) AS avg_rating\
            FROM cnt LEFT JOIN books AS b ON cnt.book_id = b.book_id LEFT JOIN ratings AS r ON b.book_id = r.book_id\
            GROUP BY author_id\
            ORDER BY AVG(rating) DESC\
            LIMIT 1) AS at LEFT JOIN authors AS a ON at.author_id = a.author_id'
)


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Самым любимым автором стала **Джоан Роулинг** со средним рейтингом 4.3. 

In [13]:
#посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок
sql(
"\
    SELECT AVG(count)\
    FROM (SELECT r.username, COUNT(review_id)\
    FROM reviews AS r\
    WHERE r.username IN (SELECT username\
                        FROM ratings\
                        GROUP BY username\
                        HAVING COUNT(rating)>50)\
    GROUP BY r.username) AS t\
"
)

,avg
0,24.333333


В среднем активные пользователи обозревают около **24 - 25** книг.

## Выводы

В ходе изучения данных не было выявлено необоснованных пропусков иявных дубликатов.

Исходя из решенных задач были получены ответы на вопросы заказчиков и выявлены следующие данные:
* **819 книг** была выпущена после 1 января 2000 года
* **количество обзоров** и **средняя оценка** книг не коррелируют друг с другом
* больше всего книг выпустило издательство **Penguin Books** (42 книги)
* самая высокий средний рейтинг книг у **Джоан Роулинг** (4,3)
* пользователи, которые поставили больше 50 оценок, имеют в среднем **24 - 25 обзоров**